In [ ]:
#import libraries
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno
from warnings import filterwarnings
filterwarnings('ignore')

In [ ]:
train=pd.read_csv('../input/datasets-for-churn-telecom/cell2celltrain.csv')
test=pd.read_csv('../input/datasets-for-churn-telecom/cell2cellholdout.csv')

In [ ]:
test.info()

In [ ]:
train.info()

In [ ]:
#categoric features
for i in train.columns:
      if train[i].dtype=='object':
            print(pd.DataFrame(train[i].value_counts()))

## Handling Missing Values

In [ ]:
train['HandsetPrice'] = train['HandsetPrice'].replace('Unknown', np.nan)
test['HandsetPrice'] = test['HandsetPrice'].replace('Unknown', np.nan)

In [ ]:
train['HandsetPrice']  = pd.to_numeric(train['HandsetPrice'] )
test['HandsetPrice']  = pd.to_numeric(test['HandsetPrice'] )

In [ ]:
print(train.select_dtypes(exclude=['int', 'float']).columns,"\n***\n"*5,train.select_dtypes(include=['int', 'float']).columns)

In [ ]:
print(train.columns[train.isnull().any()],"\n***\n"*5,test.columns[test.isnull().any()])

In [ ]:
test.columns[test.isnull().any()]

In [ ]:
train_missings=train.filter(['MonthlyRevenue', 'MonthlyMinutes', 'TotalRecurringCharge',
       'DirectorAssistedCalls', 'OverageMinutes', 'RoamingCalls',
       'PercChangeMinutes', 'PercChangeRevenues', 'ServiceArea', 'Handsets',
       'HandsetModels', 'CurrentEquipmentDays', 'AgeHH1', 'AgeHH2',
       'HandsetPrice'], axis=1)
test_missings=test.filter(['MonthlyRevenue', 'MonthlyMinutes', 'TotalRecurringCharge',
       'DirectorAssistedCalls', 'OverageMinutes', 'RoamingCalls',
       'PercChangeMinutes', 'PercChangeRevenues', 'ServiceArea', 'AgeHH1',
       'AgeHH2', 'HandsetPrice'],axis=1)

In [ ]:
msno.matrix(train_missings);

In [ ]:
msno.heatmap(train_missings,cmap='coolwarm');

In [ ]:
msno.dendrogram(train_missings);

In [ ]:
msno.bar(train_missings);

In [ ]:
train_missings.describe().T

In [ ]:
def values_table(train_missings):
        mis_val = train_missings.isnull().sum()
        mis_val_percent = 100 * train_missings.isnull().sum() / len(train_missings)
        mean=train_missings.mean()
        median=train_missings.median()
        total=train_missings.count()
        zeros=train_missings[train_missings==0].count()
        table = pd.concat([ mis_val,mis_val_percent, mean, median, total, zeros], axis=1)
        table = table.rename(columns = {0 :'Train Missing Values', 1 : '% of Train Total Values', 2 : 'Mean'
                                       , 3 : 'Median' , 4 : '# of Values' , 5 : '# of Zeros'})
        table['Data Type'] = train_missings.dtypes
        table = table[table.iloc[:,1] != 0].sort_values('% of Train Total Values', ascending=False).round(1)
        print ("Dataframe'in içinde " + str(train.shape[1]) + " sütun ve " + str(train.shape[0]) + " satır vardır.\n"      
            "Bu sütunlardan " + str(table.shape[0]) +" tanesinde missing value bulunmaktadır.")
        return table

values_table(train_missings)

In [ ]:
def values_table(test_missings):
        mis_val = test_missings.isnull().sum()
        mis_val_percent = 100 * test_missings.isnull().sum() / len(test_missings)
        mean=test_missings.mean()
        median=test_missings.median()
        total=test_missings.count()
        zeros=test_missings[test_missings==0].count()
        table = pd.concat([ mis_val,mis_val_percent, mean, median, total, zeros], axis=1)
        table = table.rename(columns = {0 :'Test Missing Values', 1 : '% of Test Total Values', 2 : 'Mean'
                                       , 3 : 'Median' , 4 : '# of Values' , 5 : '# of Zeros'})
        table['Data Type'] = test_missings.dtypes
        table = table[table.iloc[:,1] != 0].sort_values('% of Test Total Values', ascending=False).round(1)
        print ("Dataframe'in içinde " + str(train.shape[1]) + " sütun ve " + str(train.shape[0]) + " satır vardır.\n"      
            "Bu sütunlardan " + str(table.shape[0]) +" tanesinde missing value bulunmaktadır.")
        return table

values_table(test_missings)

In [ ]:
train_1=train.copy()
test_1=test.copy()

In [ ]:
train_1.Churn.replace(('Yes', 'No'), (1, 0), inplace=True)

In [ ]:
train_1['MonthlyRevenue'].fillna(train_1['MonthlyRevenue'].median(), inplace=True)
test_1['MonthlyRevenue'].fillna(test_1['MonthlyRevenue'].median(), inplace=True)

In [ ]:
train_1[train_1['Handsets'].isna()][['HandsetModels','Handsets','CurrentEquipmentDays']] 
#HandsetModels ve Handsets numeric gözükmesine rağmen kategorik değişkenler.Bu yüzden mode ile doldurmak daha doğru

In [ ]:
print("Handsets:",train_1.Handsets.mode()[0])
print("HandsetModels:",train_1.HandsetModels.mode()[0])
print("CurrentEquipmentDays:",train_1.CurrentEquipmentDays.mode()[0])

In [ ]:
train_1['Handsets'].fillna(train_1['Handsets'].mode()[0],inplace=True)

In [ ]:
train_1['HandsetModels'].fillna(train_1['HandsetModels'].mode()[0],inplace=True)

In [ ]:
train_1['CurrentEquipmentDays'].mode()[0]

In [ ]:
train_1['CurrentEquipmentDays'].value_counts()

In [ ]:
train_1['CurrentEquipmentDays'].fillna(train_1['CurrentEquipmentDays'].median(),inplace=True)

In [ ]:
train_1['ServiceArea'].fillna(train_1['ServiceArea'].mode()[0],inplace=True)
test_1['ServiceArea'].fillna(test_1['ServiceArea'].mode()[0],inplace=True)

In [ ]:
train_2=train.HandsetPrice.groupby([train['HandsetModels'],
        train['Handsets']]).sum().astype(int).reset_index(name='count_price')
display(train_2[train_2['count_price']==0])

In [ ]:
train_3=train_1.HandsetPrice.isnull().groupby([train_1['HandsetModels'],
        train_1['Handsets']]).sum().astype(int).reset_index(name='count_missing_price')

display(train_3.count_missing_price.sum())
display(train_3[train_3['count_missing_price'] > 0])


In [ ]:
train_1.loc[train_1.HandsetModels.eq(1),'HandsetPrice'] = (train_1.loc[train_1.HandsetModels.eq(1),
                            'HandsetPrice'].fillna(train_1.HandsetPrice.median()))

In [ ]:
train_1['HandsetPrice']= train_1['HandsetPrice'].fillna(train_1.groupby([train_1['HandsetModels'],
        train_1['Handsets']])['HandsetPrice'].transform('median'))

In [ ]:
train_1.HandsetPrice.isnull().sum()

In [ ]:
test_2=test.HandsetPrice.groupby([test['HandsetModels'],
        test['Handsets']]).sum().astype(int).reset_index(name='count_price')
display(test_2[test_2['count_price']==0])

In [ ]:
test_3=test_1.HandsetPrice.isnull().groupby([test_1['HandsetModels'],
        test_1['Handsets']]).sum().astype(int).reset_index(name='count_missing_price')

display(test_3.count_missing_price.sum())
display(test_3[test_3['count_missing_price'] > 0])

In [ ]:
test_1.loc[test_1.HandsetModels.eq(1),'HandsetPrice'] = (test_1.loc[test_1.HandsetModels.eq(1),
                            'HandsetPrice'].fillna(test_1.HandsetPrice.median()))

In [ ]:
test_1['HandsetPrice']= test_1['HandsetPrice'].fillna(test_1.groupby([test_1['HandsetModels'],
        test_1['Handsets']])['HandsetPrice'].transform('median'))

In [ ]:
test_1.HandsetPrice.isnull().sum()

In [ ]:
train_1['AgeHH1'].fillna(value=0,inplace=True)
test_1['AgeHH1'].fillna(value=0,inplace=True)

In [ ]:
train_1['AgeHH2'].fillna(value=0,inplace=True)
test_1['AgeHH2'].fillna(value=0,inplace=True)

In [ ]:
display(train_1.groupby(['Churn'],as_index=False)['MonthlyMinutes'].median().style.hide_index())

In [ ]:
train_1['MonthlyMinutes'].fillna(train_1.groupby('Churn')['MonthlyMinutes'].transform('median'), inplace=True)

In [ ]:
test_1['MonthlyMinutes'].fillna(test_1['MonthlyMinutes'].median(), inplace=True)

In [ ]:
train_1['TotalRecurringCharge'].fillna(train_1['TotalRecurringCharge'].median(),inplace=True)
test_1['TotalRecurringCharge'].fillna(test_1['TotalRecurringCharge'].median(),inplace=True)

In [ ]:
train_1['PercChangeRevenues'].fillna(value=0,inplace=True)
test_1['PercChangeRevenues'].fillna(value=0,inplace=True)

In [ ]:
train_1['PercChangeMinutes'].fillna(value=0,inplace=True)
test_1['PercChangeMinutes'].fillna(value=0,inplace=True)

In [ ]:
train_1['RoamingCalls'].fillna(value=0,inplace=True)
test_1['RoamingCalls'].fillna(value=0,inplace=True)

In [ ]:
train_1['OverageMinutes'].fillna(value=0,inplace=True)
test_1['OverageMinutes'].fillna(value=0,inplace=True)

In [ ]:
train_1['DirectorAssistedCalls'].fillna(value=0,inplace=True)
test_1['DirectorAssistedCalls'].fillna(value=0,inplace=True)

In [ ]:
test_1.isnull().sum()

# OUTLIER HANDLING

In [ ]:
train_yeni=train_1.select_dtypes(include=['int', 'float'])
from scipy.stats import zscore
from scipy import stats 

In [ ]:
#Z Score
train_1_replaced = train_yeni.copy()
outliers={}
for col in train_yeni:
    z_score = zscore(train_yeni[col])
    upper_replace_value = train_yeni[z_score<=3][col].max()
    lower_replace_value = train_yeni[z_score>=-3][col].min()
    train_1_replaced = train_yeni.copy()
    outliers[col] = len(train_1_replaced[col][(train_1_replaced[col] > upper_replace_value)| (train_1_replaced[col] < lower_replace_value)])

In [ ]:
outliers_zscore=pd.DataFrame(outliers.items(),columns=['Feature','Outliers'])
print("Outlier Z Score:\n",outliers_zscore)

In [ ]:
#IQR outliers
train_2_replaced = train_yeni.copy()
outliers_1={}
for col in train_yeni:
    q1 = train_yeni[col].quantile(q=0.15)
    q3 = train_yeni[col].quantile(q=0.85)
    iqr = stats.iqr(train_yeni[col]) 
    train_2_replaced = train_yeni.copy()
    upper_limit = q3 + 1.5*iqr
    lower_limit = q1 - 1.5*iqr
    outliers_1[col] = len(train_2_replaced[col][(train_2_replaced[col] > upper_limit)| (train_2_replaced[col] < lower_limit)])

In [ ]:
outliers_iqr=pd.DataFrame(outliers_1.items(),columns=['Feature','Outliers'])
print("IQR Outliers:\n",outliers_iqr)

In [ ]:
#Hard Edge Outliers
train_3_replaced = train_yeni.copy()
outliers_2={}
for col in train_yeni:
    lower_lim = train_yeni[col].quantile(q=0.01)
    upper_lim = train_yeni[col].quantile(q=0.99)
    outliers_2[col] = len(train_3_replaced[col][(train_3_replaced[col] > upper_lim)| (train_3_replaced[col] < lower_lim)])

In [ ]:
outliers_hard=pd.DataFrame(outliers_2.items(),columns=['Feature','Outliers'])
print("Hard Edge Outliers:\n",outliers_hard)

In [ ]:
numerik = train_1.select_dtypes(include=['int', 'float'])

In [ ]:
numerik

In [ ]:
# https://www.kaggle.com/kevinarvai/outlier-detection-practice-uni-multivariate#Nonparametric-methods:-Multivariate

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pylab import rcParams

import sklearn
from sklearn.cluster import DBSCAN
from collections import Counter

In [ ]:
%matplotlib inline
rcParams['figure.figsize']= 5, 4
sns.set_style('whitegrid')

In [ ]:
train_4_replaced = train_yeni.copy()

In [ ]:
model = DBSCAN(eps = 0.8, min_samples=19).fit(train_yeni)
print(model)

In [ ]:
outliers_df = pd.DataFrame(train_yeni)
print (Counter(model.labels_))
print("***"*20)
print (outliers_df[model.labels_==-1])

In [ ]:
#train_yeni.drop(['MonthlyRevenue','MonthlyMinutes','TotalRecurringCharge'], axis=1,inplace=True)

In [ ]:
fig = plt.figure()
ax = fig.add_axes([3,3,3,3])
colors = model.labels_
ax.scatter(outliers_df.iloc[outliers_df.index[:],:],outliers_df.iloc[:,:], s=100)
ax.set_xlabel('MonthlyRevenue')
ax.set_ylabel('MonthlyMinutes')
plt.title('DBSCAN outliers')

In [ ]:
train_yeni.loc[train_yeni.index[1:10000]]

In [ ]:
#Heatmap
plt.rcParams["figure.figsize"] = (30,20)
sns.heatmap(train[:].corr(),annot = True,fmt='.1g',linecolor='white',cmap="YlGnBu",linewidths=.5)
plt.title("Heatmap",fontsize= 18)
plt.show()

In [ ]:
((numerik['PercChangeMinutes']<0)==True).sum()

In [ ]:
((numerik['RoamingCalls']>555)==True).sum()

In [ ]:
((numerik['MonthlyMinutes']>2000)==True).sum()

In [ ]:
# MonthlyRevenueaylık gelir çok yüksek olabilirr o yüzden ellemedim
# PercChangeMinutes z skoru +-3 olanlar atılabilir bilgi kaybı min olacakatır.
# MonthlyMinutes çok absürt anomali olduğunu düşnmüyorum burayı
# TotalRecurringCharge normal
# DirectorAssistedCalls normal 
# OverageMinutes biraz abartı var 1500+ çıkarılabilir
# RoamingCalls 200+ silinebilir
# PercChangeMinutes normal

# Outliers Sefa

In [ ]:
#'MonthlyRevenue', 'MonthlyMinutes', 'TotalRecurringCharge',
#       'DirectorAssistedCalls', 'OverageMinutes', 'RoamingCalls',
#       'PercChangeMinutes'

In [ ]:
## BOXPLOT

In [ ]:
sns.boxplot(train['MonthlyRevenue'])

In [ ]:
sns.boxplot(train['MonthlyMinutes'])

In [ ]:
sns.boxplot(train['TotalRecurringCharge'])

In [ ]:
sns.boxplot(train['DirectorAssistedCalls'])

In [ ]:
sns.boxplot(train['OverageMinutes'])

In [ ]:
sns.boxplot(train['RoamingCalls'])

In [ ]:
sns.boxplot(train['PercChangeMinutes'])

In [ ]:
# HİSTOGRAM

In [ ]:
def plot_distro(train, col = '', y_limit=None, x_limit_r=None, x_limit_l = None):
    plt.figure(figsize=(10,10))
    plt.grid(True)
    sns.distplot(train[col][train.Churn == 'Yes'])
    sns.distplot(train[col][train.Churn == 'No'])
    plt.legend(['1(Yes)', '0(No)'])
    if y_limit:
        plt.ylim(top=y_limit)
    if x_limit_r:
        plt.xlim(right=x_limit_r)
    if x_limit_l:
        plt.xlim(left=x_limit_l)

In [ ]:
plot_distro(train, col='MonthlyMinutes', x_limit_r=6000, x_limit_l=0)

In [ ]:
plot_distro(train, col='MonthlyRevenue', x_limit_r=500, x_limit_l=0)

In [ ]:
plot_distro(train, col='TotalRecurringCharge', x_limit_r=250, x_limit_l=0)

In [ ]:
plot_distro(train, col='DirectorAssistedCalls', x_limit_r=20, x_limit_l=0)

In [ ]:
plot_distro(train, col='OverageMinutes', x_limit_r=250, x_limit_l=0)

In [ ]:
plot_distro(train, col='RoamingCalls', x_limit_r=200, x_limit_l=0)

In [ ]:
plot_distro(train, col='PercChangeMinutes', x_limit_r=1000, x_limit_l=0)

# Modelling

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
from sklearn.preprocessing import scale 
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import roc_auc_score,roc_curve
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier, VotingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler  
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB

from warnings import filterwarnings
filterwarnings('ignore')

In [ ]:
Y = numerik['Churn']
X = numerik.drop(columns=['Churn'])
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4, random_state=44)

In [ ]:
print('X train shape: ', X_train.shape)
print('Y train shape: ', y_train.shape)
print('X test shape: ', X_test.shape)
print('Y test shape: ', y_test.shape)

# Random forest

In [ ]:
rf_model = RandomForestClassifier(random_state = 42, n_estimators=150, max_depth=9, max_leaf_nodes=9).fit(X_train, y_train)
y_pred = rf_model.predict(X_test)
y_pred_train = rf_model.predict(X_train)
print("** Karmaşıklık Matrisi **\n",confusion_matrix(y_test, y_pred))
print("** Test Seti Doğruluk Oranı **\n:",accuracy_score(y_test, y_pred))
print("** Train Seti Doğruluk Oranı **\n:",accuracy_score(y_train, y_pred_train))
print(classification_report(y_test, y_pred)) 

In [ ]:
#svc = SVC(probability=True,kernel='linear')
#adb_model = AdaBoostClassifier(base_estimator = svc, random_state = 42).fit(X_train, y_train)
#y_pred = adb_model.predict(X_test)
#y_pred_train = adb_model.predict(X_train)
#print("** Karmaşıklık Matrisi **\n",confusion_matrix(y_test, y_pred))
#print("** Test Seti Doğruluk Oranı **\n:",accuracy_score(y_test, y_pred))
#print("** Train Seti Doğruluk Oranı **\n:",accuracy_score(y_train, y_pred_train))
#print(classification_report(y_test, y_pred)) 

In [ ]:
gbm_model = GradientBoostingClassifier(random_state=42).fit(X_train, y_train)
y_pred = gbm_model.predict(X_test)
y_pred_train = gbm_model.predict(X_train)
print("** Karmaşıklık Matrisi **\n",confusion_matrix(y_test, y_pred))
print("** Test Seti Doğruluk Oranı **\n:",accuracy_score(y_test, y_pred))
print("** Train Seti Doğruluk Oranı **\n:",accuracy_score(y_train, y_pred_train))
print(classification_report(y_test, y_pred)) 

In [ ]:
lgbm_model = LGBMClassifier(random_state = 42).fit(X_train, y_train)
y_pred = lgbm_model.predict(X_test)
y_pred_train = lgbm_model.predict(X_train)
print("** Karmaşıklık Matrisi **\n",confusion_matrix(y_test, y_pred))
print("** Test Seti Doğruluk Oranı **\n:",accuracy_score(y_test, y_pred))
print("** Train Seti Doğruluk Oranı **\n:",accuracy_score(y_train, y_pred_train))
print(classification_report(y_test, y_pred)) 

In [ ]:
knn = KNeighborsClassifier()
knn_model = knn.fit(X_train, y_train)
y_pred = knn_model.predict(X_test)
y_pred_train = knn_model.predict(X_train)
print("** Karmaşıklık Matrisi **\n",confusion_matrix(y_test, y_pred))
print("** Test Seti Doğruluk Oranı **\n:",accuracy_score(y_test, y_pred))
print("** Train Seti Doğruluk Oranı **\n:",accuracy_score(y_train, y_pred_train))
print(classification_report(y_test, y_pred)) 

In [ ]:
!pip install xgboost
from xgboost import XGBClassifier

In [ ]:
xgb_model = XGBClassifier().fit(X_train, y_train)

In [ ]:
xgb_model

In [ ]:
y_pred = xgb_model.predict(X_test)
accuracy_score(y_test, y_pred)

# model tuning

In [ ]:
xgb_model

In [ ]:
?xgb_model

In [ ]:
xgb_params = {
        'n_estimators': [100, 200, 500, 1000],
        'subsample': [0.6, 1.0],
        'max_depth': [3, 4, 5,6],
        'learning_rate': [0.1,0.01],
        "min_samples_split": [2,5,10]}

In [ ]:
xgb = XGBClassifier()

xgb_cv_model = GridSearchCV(xgb, xgb_params, cv = 5, n_jobs = -1, verbose = 2)

In [ ]:
xgb_cv_model.fit(X_train, y_train)

In [ ]:
xgb_cv_model.best_params_

In [ ]:
xgb = XGBClassifier(learning_rate = 0.01, 
                    max_depth = 6,
                    min_samples_split = 2,
                    n_estimators = 1000,
                    subsample = 1.0)

In [ ]:
xgb_tuned =  xgb.fit(X_train,y_train)

In [ ]:
y_pred = xgb_tuned.predict(X_test)
xgbm_model_skor = accuracy_score(y_test, y_pred)

In [ ]:
print("** Karmaşıklık Matrisi **\n",confusion_matrix(y_test, y_pred))
print("** Test Seti Doğruluk Oranı **\n:",accuracy_score(y_test, y_pred))
print("** Train Seti Doğruluk Oranı **\n:",accuracy_score(y_train, y_pred_train))

In [ ]:
xgbm_model_skor

In [ ]:
0.7284881727802537

In [ ]:
modeller = [
    knn_model,
    rf_model,
    gbm_model,
    lgbm_model,
    
]


for model in modeller:
    isimler = model.__class__.__name__
    y_pred = model.predict(X_test)
    dogruluk = accuracy_score(y_test, y_pred)
    print("-"*28)
    print(isimler + ":" )
    print("Accuracy: {:.4%}".format(dogruluk))

In [ ]:
sonuc = []

sonuclar = pd.DataFrame(columns= ["Modeller","Accuracy"])

for model in modeller:
    isimler = model.__class__.__name__
    y_pred = model.predict(X_test)
    dogruluk = accuracy_score(y_test, y_pred)    
    sonuc = pd.DataFrame([[isimler, dogruluk*100]], columns= ["Modeller","Accuracy"])
    sonuclar = sonuclar.append(sonuc)
    
    
sns.barplot(x= 'Accuracy', y = 'Modeller', data=sonuclar, color="b")
plt.xlabel('Accuracy %')
plt.title('Modellerin Doğruluk Oranları');    